# 8数据规整：聚合、合并和重塑
对于存储于不同文件、数据库的数据，进行整理。
## 8.1层次化索引
层次化索引，在一个轴上拥有两个及以上的索引。

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
data = pd.Series(np.random.randn(9),
                index = [['a','a','a','b','b','b','c','c','c'],
                        [1,2,3,1,2,3,1,2,3]])
data

a  1   -0.365814
   2    0.725251
   3    1.053213
b  1    0.156491
   2    1.361151
   3   -1.062616
c  1   -0.205297
   2    0.509454
   3   -0.632549
dtype: float64

In [4]:
data.index

MultiIndex(levels=[['a', 'b', 'c'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 1, 2, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 2]])

In [5]:
data['b']

1    0.156491
2    1.361151
3   -1.062616
dtype: float64

In [6]:
data.loc[['b','c']]

b  1    0.156491
   2    1.361151
   3   -1.062616
c  1   -0.205297
   2    0.509454
   3   -0.632549
dtype: float64

In [8]:
data.loc[:,2]

a    0.725251
b    1.361151
c    0.509454
dtype: float64

In [9]:
data.unstack()

,1,2,3
a,-0.365814,0.725251,1.053213
b,0.156491,1.361151,-1.062616
c,-0.205297,0.509454,-0.632549


In [10]:
data.unstack().stack()

a  1   -0.365814
   2    0.725251
   3    1.053213
b  1    0.156491
   2    1.361151
   3   -1.062616
c  1   -0.205297
   2    0.509454
   3   -0.632549
dtype: float64

### 8.1.1重排与分级排序

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [12]:
#索引赋名
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [14]:
frame['Colorado']

color      Green
key1 key2       
a    1         2
     2         5
b    1         8
     2        11

### 8.1.2根据级别汇总统计
DataFrame和Series进行统计的时候，有level参数，用于设置求和的级别

In [15]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [16]:
frame.sum(level='color',axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [17]:
type(frame.sum(level='color',axis=1))

pandas.core.frame.DataFrame

In [22]:
frame.sum(level='color',axis=1).iloc[0,0]

2

In [25]:
frame.sum(level='color',axis=1).loc['a','Green']

key2
1    2
2    8
Name: Green, dtype: int32

### 8.1.3使用DataFrame的列进行索引
使用DataFrame的set_index函数将其中一个或多个列转换为行索引。

In [26]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [30]:
frame2 = frame.set_index(['c','d'],drop=True)
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

## 8.2合并数据集
* pandas.merge函数，根据一个或多个健将不同的FataFrame连接起来。
* pandas.concat函数，将多个对象沿着一个轴堆砌起来。
* pandas.combine.first函数，可以将重复数据编结在一起。用一个对象的值填充另一个对象的缺失值。
### 8.2.1数据库风格的DataFrame合并

In [32]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1


,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [33]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [35]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [38]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

In [39]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [40]:
pd.merge(df1, df2, how='outer') #outer使用两个表的所有健

,key,data1,data2
0,b,0.0,1.0
1,b,0.0,3.0
2,b,1.0,1.0
3,b,1.0,3.0
4,b,5.0,1.0
5,b,5.0,3.0
6,a,2.0,0.0
7,a,2.0,2.0
8,a,4.0,0.0
9,a,4.0,2.0


### 8.2.2索引上的合并

In [43]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print(left1)
print(right1)
pd.merge(left1, right1, left_on='key', right_index=True)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [44]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [46]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
print(left2)
print(right2)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


In [47]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [48]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [49]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


### 8.2.3轴向连接
* 连接：concatenation
* 绑定：binding
* 堆叠：stacking

In [50]:
#concatenation
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [51]:
np.concatenate([arr,arr * 2],axis=1)

array([[ 0,  1,  2,  3,  0,  2,  4,  6],
       [ 4,  5,  6,  7,  8, 10, 12, 14],
       [ 8,  9, 10, 11, 16, 18, 20, 22]])

In [52]:
np.concatenate([arr,arr * 2],axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  2,  4,  6],
       [ 8, 10, 12, 14],
       [16, 18, 20, 22]])

In [55]:
df = pd.DataFrame(arr,index=['a','b','c'],columns=['x','y','z','o'])
df

,x,y,z,o
a,0,1,2,3
b,4,5,6,7
c,8,9,10,11


In [57]:
pd.concat([df,df*2],axis=0)

,x,y,z,o
a,0,1,2,3
b,4,5,6,7
c,8,9,10,11
a,0,2,4,6
b,8,10,12,14
c,16,18,20,22


In [58]:
pd.concat([df,df*2],axis=1)

,x,y,z,o,x,y,z,o
a,0,1,2,3,0,2,4,6
b,4,5,6,7,8,10,12,14
c,8,9,10,11,16,18,20,22


In [61]:
pd.concat([df,df*2],axis=1,join='inner')

,x,y,z,o,x,y,z,o
a,0,1,2,3,0,2,4,6
b,4,5,6,7,8,10,12,14
c,8,9,10,11,16,18,20,22


### 8.2.4合并重叠数据

In [63]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print(a)
print(b)
np.where(pd.isnull(a),b,a)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64


array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [64]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

## 8.3重塑和轴向旋转
* 重塑 reshape
* 轴向旋转 pivot
### 8.3.1重塑层次化索引
* stack ： 将数据的列旋转为行
* unstack ： 将数据的行旋转为列

In [3]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index=pd.Index(['ohio','colorado'],name = 'state'),
                   columns=pd.Index(['one','two','three'],name = 'number'))
data

number,one,two,three
state,,,
ohio,0,1,2
colorado,3,4,5


In [9]:
result = data.stack(0)
result #一个层次化索引的Series

state     number
ohio      one       0
          two       1
          three     2
colorado  one       3
          two       4
          three     5
dtype: int32

In [11]:
result.unstack('number')

number,one,two,three
state,,,
ohio,0,1,2
colorado,3,4,5


In [10]:
result.unstack('state')

state,ohio,colorado
number,,
one,0,3
two,1,4
three,2,5


In [12]:
#Series
s1 = pd.Series(np.arange(4),index=['a','b','c','d'])
s1

a    0
b    1
c    2
d    3
dtype: int32

In [13]:
s2 = pd.Series(np.arange(4,7),index=['c','d','e'])
s2

c    4
d    5
e    6
dtype: int32

In [14]:
data2 = pd.concat([s1,s2],keys=['one','two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int32

In [15]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [16]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [17]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

### 8.3.2将“长格式”旋转为“宽格式”

In [18]:
data = pd.read_csv('macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [20]:
periods = pd.PeriodIndex(year=data.year,
                         quarter = data.quarter,
                        name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [21]:
columns = pd.Index(['realgdp','infl','unemp'],name = 'item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [22]:
data = data.reindex(columns = columns)

In [23]:
data

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
5,2834.390,0.14,5.2
6,2839.022,2.70,5.6
7,2802.616,1.21,6.3
8,2819.264,-0.40,6.8
9,2872.005,1.47,7.0


In [24]:
data.index = periods.to_timestamp('D','end')

In [25]:
ldata = data.stack().reset_index().reset_index().rename(columns={0:'value'})


In [26]:
pivoted = ldata.pivot('date','item','value')

In [27]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
1960-06-30 23:59:59.999999999,0.14,2834.390,5.2
1960-09-30 23:59:59.999999999,2.70,2839.022,5.6
1960-12-31 23:59:59.999999999,1.21,2802.616,6.3
1961-03-31 23:59:59.999999999,-0.40,2819.264,6.8


### 8.3.3 将宽格式旋转为长格式
* pandas.melt()

In [28]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [29]:
melted = pd.melt(df, ['key'])  #
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [30]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [31]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [32]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
